In [1]:
!pip install opencv-python

In [2]:
!pip install lorem-text

In [3]:
!pip install stegano

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 49.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0


In [4]:
!pip install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00


In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from typing import List, Dict
from tensorflow import keras
from keras import models, layers
from sklearn.model_selection import train_test_split

from lorem_text import lorem
from stegano import lsb
import efficientnet.tfkeras as efn

In [2]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c alaska2-image-steganalysis

100% 30.0G/30.0G [13:40<00:00, 35.1MB/s]
100% 30.0G/30.0G [13:41<00:00, 39.2MB/s]


In [4]:
!unzip /content/alaska2-image-steganalysis.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/UERD/74675.jpg  
  inflating: /content/UERD/74676.jpg  
  inflating: /content/UERD/74677.jpg  
  inflating: /content/UERD/74678.jpg  
  inflating: /content/UERD/74680.jpg  
  inflating: /content/UERD/74681.jpg  
  inflating: /content/UERD/74682.jpg  
  inflating: /content/UERD/74683.jpg  
  inflating: /content/UERD/74684.jpg  
  inflating: /content/UERD/74685.jpg  
  inflating: /content/UERD/74686.jpg  
  inflating: /content/UERD/74687.jpg  
  inflating: /content/UERD/74688.jpg  
  inflating: /content/UERD/74689.jpg  
  inflating: /content/UERD/74690.jpg  
  inflating: /content/UERD/74691.jpg  
  inflating: /content/UERD/74692.jpg  
  inflating: /content/UERD/74693.jpg  
  inflating: /content/UERD/74694.jpg  
  inflating: /content/UERD/74695.jpg  
  inflating: /content/UERD/74696.jpg  
  inflating: /content/UERD/74697.jpg  
  inflating: /content/UERD/74698.jpg  
  inflating: /content/UERD/74699.jpg  
  inflating: 

In [ ]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

EPOCHS = 60
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
print(BATCH_SIZE)

In [ ]:
SHAPE = (512, 512, 3)

with strategy.scope():
    model = tf.keras.Sequential([
        efn.EfficientNetB3(
            input_shape=SHAPE,
            weights='imagenet',
            include_top=False
        ),
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy']
    )
    model.summary()

In [ ]:
def get_paths(directory, start_index, end_index):
    image_paths = []
    for filename in os.listdir(directory)[start_index:end_index]:  # Select images based on start and end index
        image_path = os.path.join(directory, filename)
        image_paths.append(image_path)
    return image_paths

# Cover paths - 75,000 images from the Cover directory, all labeled 0
cover_paths = np.array(get_paths('/content/Cover', 0, 75000))
cover_labels = np.array([0 for _ in cover_paths])

# Here i took 75k images stego for balancing dataset
# Jmipod 0-25k images, juniward 25k-50k images and uerd 50k-75k images
jmipod_paths = get_paths('/content/JMiPOD', 0, 25000)
juniward_paths = get_paths('/content/JUNIWARD', 25000, 50000)
uerd_paths = get_paths('/content/UERD', 50000, 75000)

# combine stego paths
stego_paths = np.array(jmipod_paths + juniward_paths + uerd_paths)
stego_labels = np.array([1 for _ in stego_paths])


In [ ]:
print(
    cover_paths.shape,
    cover_labels.shape,
    stego_paths.shape,
    stego_labels.shape
)

In [ ]:
img_paths = np.concatenate((cover_paths, stego_paths), axis=None)
img_labels = np.concatenate((cover_labels, stego_labels), axis=None)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(img_paths, img_labels, test_size=0.15, shuffle=True)
X_train_paths, X_val_paths, y_train, y_val = train_test_split(X_train_paths, y_train, test_size=0.15, shuffle=True)

In [ ]:
msg = "Tamaños set {}. X: {}, y: {}"
print(msg.format("train", len(X_train_paths), len(y_train)))
print(msg.format("test", len(X_test_paths), len(y_test)))
print(msg.format("validation", len(X_val_paths), len(y_val)))

In [ ]:
print(X_train_paths[:5])
print(y_train[:5])

In [ ]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

def decode_img(path, label):
    bits = tf.io.read_file(path)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

def build_dataset(X, y):
    return tf.data.Dataset.from_tensor_slices((X, y)) \
        .map(decode_img, num_parallel_calls=AUTO) \
        .map(data_augment, num_parallel_calls=AUTO) \
        .batch(BATCH_SIZE) \
        .prefetch(AUTO)

train_ds = build_dataset(X_train_paths, y_train)
test_ds = build_dataset(X_test_paths, y_test)
val_ds = build_dataset(X_val_paths, y_val)


In [ ]:
print(f"Training dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")

In [ ]:
train_ds

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=val_ds
)

In [ ]:
# finding the best epoch and accuracy
best_epoch = np.argmax(history.history['accuracy']) + 1
best_accuracy = max(history.history['accuracy'])*100

# finding the best epoch and val_accuracy
best_epoch_val = np.argmax(history.history['val_accuracy']) + 1
best_accuracy_val = max(history.history['val_accuracy'])*100

print(f"Best Value Accuracy: {best_accuracy_val:.4f} at Epoch {best_epoch_val}")

print(f"Best Accuracy: {best_accuracy:.4f} at Epoch {best_epoch}")

In [ ]:
# If you're using a strategy, make sure to save the model under it.
with strategy.scope():
    model.save("model_25_epoch.h5")


In [ ]:
print(f"Training dataset size: {len(train_ds)}")
print(f"Validation dataset size: {len(val_ds)}")

Training dataset size: 6774
Validation dataset size: 1196


In [ ]:
loaded_model = load_model("/content/model_25_epoch.h5", compile=False)
loaded_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b3 (Functional)    │ (None, 16, 16, 1536)   │    10,783,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,785,065 (41.14 MB)

 Trainable params: 10,697,769 (40.81 MB)

 Non-trainable params: 87,296 (341.00 KB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model architecture and weights
model_path = "/content/model_25_epoch.keras"
model = load_model(model_path)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

# Resume training
initial_epoch = 25
total_epochs = 60
 
try:
    history = model.fit(
        train_ds,
        epochs=total_epochs,
        initial_epoch=initial_epoch,
        validation_data=val_ds
    )
except Exception as e:
    print(f"An error occurred: {e}")


Epoch 26/60


I0000 00:00:1730109140.834484     111 service.cc:145] XLA service 0x7ee0a0002800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730109140.834577     111 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1730109239.591894     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3570s 504ms/step - accuracy: 0.6652 - loss: 0.5744 - val_accuracy: 0.6244 - val_loss: 0.6378
Epoch 27/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3331s 492ms/step - accuracy: 0.6647 - loss: 0.5752 - val_accuracy: 0.6679 - val_loss: 0.5705
Epoch 28/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3326s 491ms/step - accuracy: 0.6697 - loss: 0.5679 - val_accuracy: 0.6350 - val_loss: 0.6585
Epoch 29/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3325s 491ms/step - accuracy: 0.6762 - loss: 0.5607 - val_accuracy: 0.6774 - val_loss: 0.5603
Epoch 30/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3324s 491ms/step - accuracy: 0.6818 - loss: 0.5560 - val_accuracy: 0.6730 - val_loss: 0.5731
Epoch 31/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3325s 491ms/step - accuracy: 0.6887 - loss: 0.5491 - val_accuracy: 0.6770 - val_loss: 0.5631
Epoch 32/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3326s 491ms/step - accuracy: 0.6918 - loss: 0.5429 - val_accuracy: 0.6553 - val_loss: 0.5912
Epoch 33/60
6774/6774 ━━━━━━━━━━━━━━━━━━━━ 3326s 491ms/step 


KeyboardInterrupt



In [ ]:
# Save model weights
model.save_weights("model_weights_35_epoch.weights.h5")


In [ ]:
try:
    model.save("model_35_epoch.keras")
    print("Model saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the model: {e}")


Model saved successfully.


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

model_path = "/content/model_43_epoch.keras"
model = load_model(model_path)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

checkpoint_callback = ModelCheckpoint(
    "model_checkpoint.keras",
    save_best_only=True,     # Only save if validation loss improves
    monitor='val_loss',
    mode='min',
    verbose=1
)


history = model.fit(
    train_ds,
    epochs=60,
    initial_epoch=43,
    validation_data=val_ds,
    callbacks=[checkpoint_callback]
)


In [ ]:
# If you're using a strategy, make sure to save the model under it.
with strategy.scope():
    model.save("model__epoch.keras")

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model

# Load the model
# model = load_model("/kaggle/input/mymodel25/tensorflow2/default/1/model_25_epoch.h5", compile=False)
model_path = "/kaggle/working/model_43_epoch.keras"
model = load_model(model_path)

import os

# path of test directory
def get_paths(test_directory):

    return [os.path.join(test_directory, fname) for fname in os.listdir(test_directory) if fname.endswith(('.png', '.jpg', '.jpeg'))]

def get_test_dataset(test_directory):
    test_paths = get_paths(test_directory)
    test_labels = np.array([0] * len(test_paths)) # dummy labels

    test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)) \
        .map(decode_img, num_parallel_calls=AUTO) \
        .batch(BATCH_SIZE) \
        .prefetch(AUTO)

    return test_ds, test_paths

# Load the test dataset
test_directory = '/kaggle/input/alaska2-image-steganalysis/Test'
test_ds, test_paths = get_test_dataset(test_directory)

predictions = model.predict(test_ds)

submission_df = pd.DataFrame({
    'Id': [path.split('/')[-1] for path in test_paths],
    'Label': predictions.flatten()
})

submission_df.to_csv('submission1.csv', index=False)
print("Submission file created: submission.csv")


In [1]:
from tensorflow.keras.models import load_model

model_path = r"E:\Projects\Steganalysis\artifacts\Private\model_checkpoint2.keras"
model = load_model(model_path)

TypeError: <class 'keras.src.models.sequential.Sequential'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras', 'class_name': 'Sequential', 'config': {'name': 'sequential', 'trainable': True, 'dtype': 'float32', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 512, 512, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}, 'registered_name': None}, {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'efficientnet-b3', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 512, 512, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layer